In [10]:
import os, sys, pandas as pd, glob
import datetime, time
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('mode.chained_assignment', None)

In [11]:
input_dir = "../data/benign_attack/nfcapd.*.csv"
input_data_file_list = glob.glob(input_dir)

li = []
for filename in input_data_file_list:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)
# filter out the last few lines that are aggregate stats and not flow records
df = df[~pd.isna(df["pr"])]


/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (2,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [13]:
def getTimestamp(time_str):
    return time.mktime(datetime.datetime.strptime(time_str, "%Y-%m-%d %H:%M:%S").timetuple())
    
df['st_timestamp'] = df['ts'].apply(getTimestamp)
df['et_timestamp'] = df['te'].apply(getTimestamp)

In [19]:
df_mac2ip = pd.read_csv('data/iot_mac2ip.csv', sep='\t')
df_mac2ip['fname'] = df_mac2ip['MAC'].apply(lambda x: x.replace(':', '') + '.csv')
mac2ip_list = df_mac2ip.values.tolist()
dfTemp = df
dfTemp['flow_tag'] = ""
for i in range(0, len(mac2ip_list)):
    fname = '../data/annotations/' + mac2ip_list[i][3]
    print(fname)
    ip_addr = mac2ip_list[i][1]
    df_annotation = pd.read_csv(fname)
    l = df_annotation.values.tolist()
    for i in range(0, len(l)):
        v = l[i]
        (st, et) = (v[0], v[1])
        tag = v[2]
        dfTemp['flow_tag'].loc[(df['st_timestamp'] >= st) & (df['st_timestamp'] < et) & ((df['sa'] == ip_addr) | (df['da'] == ip_addr)) ] = tag

    

../data/annotations/ec1a59832811.csv
../data/annotations/ec1a5979f489.csv
../data/annotations/00166cab6b88.csv
../data/annotations/50c7bf005639.csv
../data/annotations/70ee50183443.csv
../data/annotations/0017882B9A25.csv
../data/annotations/44650D56CCD3.csv
../data/annotations/F4F5D88F0A3C.csv
../data/annotations/74C63B29D71D.csv
../data/annotations/D073D5018308.csv


In [20]:
def getFlowType(x):
    if x == "":
        return 0
    else:
        return 1
dfTemp["flow_type"] = dfTemp["flow_tag"].apply(getFlowType)

In [20]:
ips_to_remap_list = ["192.168.1.1", "192.168.1.2"]
ips_to_remap_dict = dict.fromkeys(ips_to_remap_list, 1)

In [8]:
## Also consider doing it for pcaps

## TODO: Tarun
## function to generate IPs

## Either assume an IP list 
## # of IP addresses 

## Otherwise generate a random set of IP addresses 
# of IPs/ Standard deviation. Pick a random mean. 

import numpy as np
import ipaddress

def generate_ips(num_ip, mode == "gen", filename = None, std_dev=1000):
    if mode == "file":
        df = pd.read_csv(filename, header=0)
        ip_list = list(df["ip"])
    elif mode == "gen":
        ip_list = generate_ips_from_normal(num_ip, std_dev)
    return ip_list
        

## assumes normally distributed IPs. Input: Standard deviation of the IP distance, # of IP addresses
def generate_ips_from_normal(num_ip, std_dev=10000):
    mean_ip = 2155905152 # 128.128.128.128
    ip_list = np.random.normal(mean_ip, std_dev, num_ip)
    ip_list_str = [ipaddress.ip_address(int(x)).__str__() for x in ip_list]
    return ip_list_str

generate_ips(100)

100


In [41]:
ip_list = get_ip_pool("data/ip_list.csv")
last_remap_idx = 0
len_ip_list = len(ip_order_list)
flow_map = {}

# remap the IP address. Don't parallelize the function
def remap_ip(x, field_name):
    src_ip, src_port, dst_ip, dst_port = (x["sa"], x["sp"], x["da"], x["dp"])
    ip_to_map = src_ip
    if field_name == "da":
        ip_to_map = dst_ip
    
    global last_remap_idx, flow_map
    flow_tuple = (src_ip, src_port, dst_ip, dst_port)
    if ip_to_map in ips_to_remap_dict:
        if flow_tuple in flow_map:
            ip_to_map = flow_map[flow_tuple]
        else:
            ip_to_map = ip_list[last_remap_idx % len_ip_list]
            flow_map[flow_tuple] = ip_to_map
            last_remap_idx += 1
    return ip_to_map

In [42]:
df["src_ip_remapped"] = df.apply(lambda x: remap_ip(x, 'sa'), axis=1)
df["dst_ip_remapped"] = df.apply(lambda x: remap_ip(x, 'da'), axis=1)
outdir = "data/remapped_data/"
df.to_csv("data/remapped_data/")

[['2018-10-26 06:00:25', '2018-10-26 06:04:35', '250.011', '192.168.1.1', '192.168.1.119', '0', 0.0, 'ICMP', '........', 0.0, 0.0, 10.0, 560.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, '0.0.0.0', '0.0.0.0', 0.0, 0.0, '00:00:00:00:00:00', '00:00:00:00:00:00', '00:00:00:00:00:00', '00:00:00:00:00:00', '0-0-0', '0-0-0', '0-0-0', '0-0-0', '0-0-0', '0-0-0', '0-0-0', '0-0-0', '0-0-0', '0-0-0', 0.0, 0.0, 0.0, '0.0.0.0', '0/0', 0.0, '1969-12-31 18:00:00.000', '192.168.10.10', '192.168.1.119'], ['2018-10-26 06:04:36', '2018-10-26 06:04:36', '0.097', '192.168.1.119', '8.8.8.8', '36258', 53.0, 'UDP', '........', 0.0, 0.0, 1.0, 32.0, 1.0, 48.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, '0.0.0.0', '0.0.0.0', 0.0, 0.0, '00:00:00:00:00:00', '00:00:00:00:00:00', '00:00:00:00:00:00', '00:00:00:00:00:00', '0-0-0', '0-0-0', '0-0-0', '0-0-0', '0-0-0', '0-0-0', '0-0-0', '0-0-0', '0-0-0', '0-0-0', 0.0, 0.0, 0.0, '0.0.0.0', '0/0', 0.0, '1969-12-31 18:00:00.000', '192.168.1.119', '8.8.8.8'], ['2018-10-

In [ ]:
## TODO Arjun: data partitioning 

# functionality. Parameter: # of partitions, split_logic
## Split logics
### maintain ratio of b2m
### split randomly  
### based on source/dst IP addresses